In [48]:
import requests # http requests
from bs4 import BeautifulSoup # Webscrape
from collections import defaultdict
import pandas as pd 

In [55]:
# CONFIG: Skills & Place of Work and pages to scrape
skill = "Registered Nurse"
place = "Berkeley CA"
no_of_pages = 20
print(f"Scraping all {skill} listings near {place} from {no_of_pages} pages of indeed")
url = None 
url = "https://www.indeed.com/jobs?q=lvn&l=Red%20Bluff%2C%20CA&vjk=4ab285f8bd95d5d3"

Scraping all Registered Nurse listings near Berkeley CA from 20 pages of indeed


In [56]:
# this was used for the person contacting me who had these details for their system
headers = {
    "User-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}


indeed_posts=[]

for page in range(no_of_pages):
        jobs = None
    
    # Connecting to India Indeed
        
        if not url:
            url = 'https://www.indeed.com/jobs?q=' + skill + \
                '&l=' + place + '&sort=date' +'&start='+ str(page * 10)
#         
        
        # Get request to indeed with headers above (you don't need headers!)
        response = requests.get(url, headers=headers)
        html = response.text

        # Scrapping the Web (you can use 'html' or 'lxml')
        soup = BeautifulSoup(html, 'lxml')

        # Outer Most Entry Point of HTML:
        outer_most_point=soup.find('div',attrs={'id': 'mosaic-provider-jobcards'})
        
        # "UL" lists where the data are stored:
        
        for i in outer_most_point.find('ul'):
            
        # Job Title:
        
            job_title=i.find('h2',{'class':'jobTitle jobTitle-color-purple jobTitle-newJob'})
    
            if job_title != None:
                jobs=job_title.find('a').text
#                 print(jobs)
            
        # Company Location:
    
            if i.find('div',{'class':'companyLocation'}) != None:
                company_location = i.find('div',{'class':'companyLocation'}).text
#                 print(company_location)
            
                
        # Company Name:
    
            if i.find('span',{'class':'companyName'}) != None:
                company=i.find('span',{'class':'companyName'}).text   
                
        # Links: these Href links will take us to full job description
        
            if i.find('a') != None:
                links=i.find('a',{'class':'jcs-JobTitle'})['href']
                links = 'https://in.indeed.com' +links
#                 print(links)
                
        # Salary if available:
        
            if i.find('div',{'class':'attribute_snippet'}) != None:
                salary=i.find('div',{'class':'attribute_snippet'}).text

            else:
                salary='N/A'

        # Job Post Date:

            if i.find('span', attrs={'class': 'date'}) != None:
                post_date = i.find('span', attrs={'class': 'date'}).text

        # Put everything together in a list of lists for the default dictionary
                        
            indeed_posts.append([company,company_location,links,salary, post_date])
            
# put together in list

# (create a dictionary with keys and a list of values from above "indeed_posts")

indeed_dict_list=defaultdict(list)


# Fields for our DF 

indeed_spec=['Company','Location','link','Salary','Job_Posted_Date']

In [57]:
# list of lists containing jobs with (company,job_title, href_link,salary,when job posted)
indeed_posts[0:1]

[['Red Bluff Healthcare Center',
  'Red Bluff, CA 96080',
  'https://in.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CYQU5hVYLhRVQJWuPQ2Y4JPsasCPqHGy7DNSLPihWRYByN6v8RVpbQGCdIF2a9YF0XNpefCflPzPvUzFX9Qa9bTflIbFG2j-zuZ-Ayg3F0RiCEuVBNUcGUyQY4gSaCS3XZEETF6DLBHzyXY-a7KsuXx9H_pSaqqQkLrPAvIlcCC5UA5aA2USdiETBTg1c0AL-qZf7VmlvppcGc2UWUDUPv5Rmr-1hob0TGTLQ3t4LaQubUPSLoJ8HpG5uqPIazzasqaaI2IJjA2dlRajE8gS3ShkcGRJS9-Psypn3sgeVN0apQIgIOiTNlwy7-DIAWAK87DOBgiouKhq4kHi9noExH0XPSdmB5h8tWi_HOWXUys3E7zPL0V0bkX4U1uikT4Ln4yURN7SaKwHCMfh1IaRgiB6MwOcm28mxT5atkvTM2tniaybsDEstS0TmXfohnreBeZnstbTb9fO3ve8l5qf6f-1STZJPBDoVyZdGoI68Alg==&xkcb=SoCw-_M3cqyLEDALcT0LbzkdCdPP&p=0&fvj=1&vjs=3',
  '$28.50 - $30.00 an hour',
  'Posted5 days ago']]

In [118]:
def clean_location(text):
    return text

def clean_salary(salary):
    if not salary.endswith("an hour"):
        salary = "N/A"
    return salary
    
def cleanup_df(df):
    data = df.to_dict(orient = "records")
#     print(data[0])
    for job in data:
        location = job.get('Location')
        salary = job.get('Salary')
        salary = clean_salary(salary)
        print(salary)
    
    return df

In [119]:
# print('These Href links will go to a new page containing full job description')
# print('\n')
# print(pd.DataFrame(indeed_posts,columns=indeed_spec)['link'][0]) 
# #these are not the same, probably from recruiter(s)
# print(pd.DataFrame(indeed_posts,columns=indeed_spec)['link'][1])

df = pd.DataFrame(indeed_posts,columns=indeed_spec)
df = cleanup_df(df)
df.to_csv("rn_berkeley.csv")
# df

$28.50 - $30.00 an hour
$25 - $28 an hour
From $26 an hour
N/A
$50 - $70 an hour
N/A
$45 an hour
$40 - $65 an hour
N/A
$35 an hour
N/A
N/A
N/A
N/A
N/A
$38 - $39 an hour
$45 an hour
N/A
$28.50 - $30.00 an hour
$25 - $28 an hour
$50 - $70 an hour
$45 an hour
N/A
N/A
N/A
N/A
$35 an hour
N/A
$38 - $39 an hour
N/A
N/A
$45 an hour
$45 an hour
$30 - $40 an hour
$21 - $25 an hour
N/A
$28.50 - $30.00 an hour
$25 - $28 an hour
From $26 an hour
N/A
$45 an hour
N/A
$50 - $70 an hour
N/A
N/A
N/A
N/A
N/A
$38 - $39 an hour
$35 an hour
N/A
$45 an hour
$45 an hour
N/A
$28.50 - $30.00 an hour
$25 - $28 an hour
From $26 an hour
$35 an hour
N/A
N/A
$50 - $70 an hour
N/A
$45 an hour
$45 an hour
$45 an hour
N/A
N/A
N/A
$21 - $25 an hour
$30 - $40 an hour
N/A
N/A
$28.50 - $30.00 an hour
$25 - $28 an hour
$50 - $70 an hour
$45 an hour
N/A
N/A
N/A
N/A
$35 an hour
N/A
N/A
N/A
$45 an hour
$45 an hour
$38 - $39 an hour
$30 - $40 an hour
$21 - $25 an hour
N/A
$28.50 - $30.00 an hour
$25 - $28 an hour
N/A
$50 - $70

In [107]:
job_descr_txt=[]

# Indeed DF with columns we made above and the stored data from scraping
Indeed_DF = pd.DataFrame(indeed_posts,columns=indeed_spec)

# convert Series to lists of strings
my_super_fun_Indeed_links = list(Indeed_DF['link'])

# iterator will be our index value for default_dict_list
for i in range(len(my_super_fun_Indeed_links)):
    
    url_href= my_super_fun_Indeed_links[i]
    print(url_href)
    response = requests.get(url_href, headers=headers)
    html_ = response.text
    soup_ = BeautifulSoup(html_, 'lxml')
    
    for ii in soup_.find('div',{'class':'jobsearch-jobDescriptionText'}):
        try:
            job_descr_txt.append([i,''.join(ii.text.strip())])
        except AttributeError:
            job_descr_txt.append([i,''])
    break
job_descr_txt

https://in.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CYQU5hVYLhRVQJWuPQ2Y4JPsasCPqHGy7DNSLPihWRYByN6v8RVpbQGCdIF2a9YF0XNpefCflPzPvUzFX9Qa9bTflIbFG2j-zuZ-Ayg3F0RiCEuVBNUcGUyQY4gSaCS3XZEETF6DLBHzyXY-a7KsuXx9H_pSaqqQkLrPAvIlcCC5UA5aA2USdiETBTg1c0AL-qZf7VmlvppcGc2UWUDUPv5Rmr-1hob0TGTLQ3t4LaQubUPSLoJ8HpG5uqPIazzasqaaI2IJjA2dlRajE8gS3ShkcGRJS9-Psypn3sgeVN0apQIgIOiTNlwy7-DIAWAK87DOBgiouKhq4kHi9noExH0XPSdmB5h8tWi_HOWXUys3E7zPL0V0bkX4U1uikT4Ln4yURN7SaKwHCMfh1IaRgiB6MwOcm28mxT5atkvTM2tniaybsDEstS0TmXfohnreBeZnstbTb9fO3ve8l5qf6f-1STZJPBDoVyZdGoI68Alg==&xkcb=SoCw-_M3cqyLEDALcT0LbzkdCdPP&p=0&fvj=1&vjs=3


[[0, 'LVN Charge Nurse'],
 [0, ''],
 [0,
  'Red Bluff Healthcare (RBHC) has immediate opening for an LVN for 12 hour the evening/Noc shift, which has a $5/hour shift differential.'],
 [0, ''],
 [0,
  'RBHC is a 58 bed skilled nursing facility that focuses on improving the health of our patients and getting them back home ASAP. Our achievement to 5 Stars tells you we meet our mission very well.'],
 [0, ''],
 [0,
  'We have a reputation for being a fun place to work. Our 2-Way radios allow great communication and teamwork.'],
 [0, ''],
 [0,
  'Please stop by and meet us to see for yourself why we are known as the facility that smiles.'],
 [0, ''],
 [0, '·'],
 [0, ''],
 [0, 'Job Types: Full-time, Part-time'],
 [0, ''],
 [0, 'Pay: $28.50 - $30.00 per hour'],
 [0, ''],
 [0, 'Schedule:'],
 [0, ''],
 [0, '12 hour shift\n8 hour shift'],
 [0, ''],
 [0, 'Supplemental Pay:'],
 [0, ''],
 [0, 'Bonus pay\nDifferential pay\nOvertime pay\nSigning bonus'],
 [0, ''],
 [0, 'License/Certification:'],
 [0,